In [18]:
keyword_raw = "flowspec"

In [121]:
import cfscrape

from lxml import etree

import pandas as pd
pd.options.display.max_colwidth = 100

import urllib

import re

from random import randint
from time import sleep

In [ ]:
page = 0 
df = pd.DataFrame()
    
while True: 
    #################################################################################
    keyword = urllib.parse.quote_plus("\""+keyword_raw+"\"")
    url="https://scholar.google.nl/scholar?start="+str(page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(url)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 10

        

https://scholar.google.nl/scholar?start=0&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=10&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=20&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=30&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=40&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=50&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=60&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=70&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=80&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=90&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=100&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.googl

sleeping...
https://scholar.google.nl/scholar?start=920&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=930&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=940&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=950&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=960&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=970&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=980&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=990&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1000&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1010&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1020&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleepin

https://scholar.google.nl/scholar?start=1820&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1830&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1840&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1850&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1860&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1870&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1880&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1890&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1900&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1910&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=1920&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...

sleeping...
https://scholar.google.nl/scholar?start=2730&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2740&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2750&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2760&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2770&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2780&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2790&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2800&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2810&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2820&q=%22flowspec%22&hl=nl&as_sdt=0,5
sleeping...
https://scholar.google.nl/scholar?start=2830&q=%22flowspec%22&hl=nl&as_sdt=0,5

In [ ]:
df

In [ ]:
import os
os.system('say "your program has finished"')

In [ ]:
df.to_csv('output_crawler_google.csv')